# Assignment 2

Before working on this assignment please read these instructions fully. In the submission area, you will notice that you can click the link to **Preview the Grading** for each step of the assignment. This is the criteria that will be used for peer grading. Please familiarize yourself with the criteria before beginning the assignment.

An NOAA dataset has been stored in the file `data/C2A2_data/BinnedCsvs_d400/fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89.csv`. This is the dataset to use for this assignment. Note: The data for this assignment comes from a subset of The National Centers for Environmental Information (NCEI) [Daily Global Historical Climatology Network](https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/readme.txt) (GHCN-Daily). The GHCN-Daily is comprised of daily climate records from thousands of land surface stations across the globe.

Each row in the assignment datafile corresponds to a single observation.

The following variables are provided to you:

* **id** : station identification code
* **date** : date in YYYY-MM-DD format (e.g. 2012-01-24 = January 24, 2012)
* **element** : indicator of element type
    * TMAX : Maximum temperature (tenths of degrees C)
    * TMIN : Minimum temperature (tenths of degrees C)
* **value** : data value for element (tenths of degrees C)

For this assignment, you must:

1. Read the documentation and familiarize yourself with the dataset, then write some python code which returns a line graph of the record high and record low temperatures by day of the year over the period 2005-2014. The area between the record high and record low temperatures for each day should be shaded.
2. Overlay a scatter of the 2015 data for any points (highs and lows) for which the ten year record (2005-2014) record high or record low was broken in 2015.
3. Watch out for leap days (i.e. February 29th), it is reasonable to remove these points from the dataset for the purpose of this visualization.
4. Make the visual nice! Leverage principles from the first module in this course when developing your solution. Consider issues such as legends, labels, and chart junk.

The data you have been given is near **Ann Arbor, Michigan, United States**, and the stations the data comes from are shown on the map below.

In [2]:
import matplotlib.pyplot as plt
import mplleaflet
import pandas as pd

def leaflet_plot_stations(binsize, hashid):

    df = pd.read_csv('data/C2A2_data/BinSize_d{}.csv'.format(binsize))

    station_locations_by_hash = df[df['hash'] == hashid]

    lons = station_locations_by_hash['LONGITUDE'].tolist()
    lats = station_locations_by_hash['LATITUDE'].tolist()

    plt.figure(figsize=(8,8))

    plt.scatter(lons, lats, c='r', alpha=0.7, s=200)

    return mplleaflet.display()

leaflet_plot_stations(400,'fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89')

ModuleNotFoundError: No module named 'mplleaflet'

## Data Prepration

In [3]:
# Get Data Set
import numpy as np
import pandas as pd
df = pd.read_csv('data/C2A2_data/BinnedCsvs_d400/fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89.csv')
# convert Date in to date time format
df['Date'] = pd.to_datetime(df['Date'])
df['Data_Value']= df['Data_Value']/10

# seprate the month, days & years
df['Month_Day'] = df['Date'].dt.strftime('%m-%d')
df['Year'] = df['Date'].dt.year

# drop 02-29
df = df[df['Month_Day']!='02-29']

# Get data for 2015
data = df[df['Year'] != 2015]
data_2015 = df[df['Year'] == 2015]

# drop extra columns
data = data.drop(['Month_Day', 'Year'],axis=1)
data_2015= data_2015.drop(['Month_Day', 'Year'],axis=1)
data.info()

FileNotFoundError: File b'data/C2A2_data/BinnedCsvs_d400/fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89.csv' does not exist

In [30]:
data.head(5)

,ID,Date,Element,Data_Value
0,USW00094889,2014-11-12,TMAX,2.2
1,USC00208972,2009-04-29,TMIN,5.6
2,USC00200032,2008-05-26,TMAX,27.8
3,USC00205563,2005-11-11,TMAX,13.9
4,USC00200230,2014-02-27,TMAX,-10.6


In [41]:
#Reindexing

d_ind = pd.DatetimeIndex(data['Date'])
data["Month"] = d_ind.month
data["Day"] = d_ind.day

# 2015
d15_ind = pd.DatetimeIndex(data_2015['Date'])
data_2015["Month"] = d15_ind.month
data_2015["Day"] = d15_ind.day


In [42]:
#The maximum temperatures records

# 2005 - 2014

max_temp = data[data["Element"]=="TMAX"].groupby(["Month", "Day"])["Data_Value"].max()
min_temp = data[data["Element"]=="TMIN"].groupby(["Month", "Day"])["Data_Value"].min()

# for 2015

max_temp_2015 = data_2015[data_2015["Element"]=="TMAX"].groupby(["Month", "Day"])["Data_Value"].max()
min_temp_2015 = data_2015[data_2015["Element"]=="TMIN"].groupby(["Month", "Day"])["Data_Value"].min()

In [43]:
#When where the records broken?
max_rec_temp = (max_temp_2015 > max_temp).as_matrix()
print(np.sum(max_rec_temp))
min_rec_temp = (min_temp_2015 < min_temp).as_matrix()
print(np.sum(min_rec_temp))

37
32


In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (20,12)
 # axis
x_axis = np.arange('2015-01-01', '2016-01-01', dtype='datetime64[D]')
x_axis = np.array(list(map(pd.to_datetime, x_axis)))


plt.title('Ann Arbor, Michigan, United States', fontsize=24)

#Past record temperatures
plt.plot(x_axis, min_temp.as_matrix(), '-d', color='blue', ms=2) 
plt.plot(x_axis, max_temp.as_matrix(), '-o', color='green', ms=2) 

#2015 record temperatures
plt.plot(x_axis[max_rec_temp], max_temp_2015[max_rec_temp].as_matrix(), 'rd',
         x_axis[min_rec_temp], min_temp_2015[min_rec_temp].as_matrix(), 'k^', 
         ms=9)

import matplotlib.dates as mdates
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%b'))
plt.gca().xaxis.set_major_locator(mdates.MonthLocator())

plt.legend(['Past Record Lows', 'Past Record Highs', "2015 record Highs", "2015 record Lows"], loc='lower left', bbox_to_anchor=(1.0, 0.5), prop={'size':16})
plt.tick_params(labelsize=18)

for spine in plt.gca().spines.values():
    spine.set_visible(False)
plt.gca().fill_between(x_axis, min_temp, max_temp, facecolor='grey', alpha=0.5)
plt.savefig('assignment.jpeg',quality=95)

NameError: name 'np' is not defined